In [1]:
# 런타임 -> 런타임 유형변경 들어가서 gpu or tpu설정
# gpu버전 graphic 확인
!nvidia-smi

Tue Sep  3 07:19:39 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |   1586MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from pprint import pprint
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier # rf분류기
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [0]:
# data load
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/data/train_1.csv')
test_data_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/data/test1_1.csv')
test_data_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/data/test2_1.csv')
survival_label = train_data['survival_time']
amount_spent_label = train_data['law_amount_spent']

In [5]:
print(train_data.shape)
print(test_data_1.shape)
print(test_data_2.shape)

(40000, 231)
(20000, 228)
(20000, 228)


In [0]:
train_data = train_data.drop('survival_time', axis=1)
train_data = train_data.drop('law_amount_spent', axis=1)
train_data = train_data.drop('acc_id', axis=1)

test_data_1 = test_data_1.drop('acc_id', axis=1)
test_data_2 = test_data_2.drop('acc_id', axis=1)

In [7]:
print(train_data.shape)
print(test_data_1.shape)
print(test_data_2.shape)

(40000, 228)
(20000, 227)
(20000, 227)


In [0]:
test_data_col = test_data_1.columns

In [0]:
#train_data = train_data.set_index(test_data_col)
train_data = pd.DataFrame(train_data, columns=test_data_col)

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, survival_label, test_size=0.2, random_state=321)

In [12]:
print(x_train.shape)
print(x_test.shape)

(32000, 227)
(8000, 227)


In [13]:
# x_train = x_train.reshape(1121283, 37,1)
# x_test = x_test.reshape(480551, 37,1)
x_train = np.array(x_train)
x_test = np.array(x_test)
print(x_train.shape)
print(x_test.shape)
x_train = x_train.reshape(32000, 227,1)
x_test = x_test.reshape(8000, 227,1)
print(x_train.shape)
print(x_test.shape)

(32000, 227)
(8000, 227)
(32000, 227, 1)
(8000, 227, 1)


In [14]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

Using TensorFlow backend.


In [15]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape[1])

65


In [16]:
# fit and evaluate a model
verbose, epochs, batch_size = 0, 100, 128
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
# fit network
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluate model
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=0)

W0903 07:23:22.908208 140659562456960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0903 07:23:22.966368 140659562456960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0903 07:23:22.975628 140659562456960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0903 07:23:23.035921 140659562456960 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0903 07:23:23.046230 

In [18]:
# x_train = x_train.reshape(20000, 37,1)
# x_test = x_test.reshape(20000, 37,1)
test_data_1 = np.array(test_data_1)
test_data_2 = np.array(test_data_2)
print(test_data_1.shape)
print(test_data_2.shape)
test_data_1 = test_data_1.reshape(20000, 227,1)
test_data_2 = test_data_2.reshape(20000, 227,1)
print(test_data_1.shape)
print(test_data_2.shape)

(20000, 227)
(20000, 227)
(20000, 227, 1)
(20000, 227, 1)


In [0]:
from sklearn.metrics import accuracy_score
predictions_1 = model.predict_classes(test_data_1)
predictions_2 = model.predict_classes(test_data_2)

for i in range(len(predictions_1)):
    if predictions_1[i] == 0:
          predictions_1[i] = 1
    if predictions_2[i] == 0:
          predictions_2[i] = 1
        
df_1 = pd.DataFrame(predictions_1)
df_2 = pd.DataFrame(predictions_2)

#df_1 = df_1.rename(columns = {'0':'survival_time'})
#df_2 = df_2.rename(columns = {'0':'survival_time'})

df_1.to_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/result/190903_test1_result1_6.csv')
df_2.to_csv('/content/drive/My Drive/Colab Notebooks/bigcontest/result/190903_test2_result1_6.csv')